In [ ]:
import tensorflow as tf
import keras
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.layers import Flatten, Input, Dense, Lambda
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image

from keras.models import Sequential, Model
from glob import glob
import cv2
from sklearn import model_selection


In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# make a vector of images with label 1 when both resembles same image and 0 when both resembles different image

imagev=[]
files= sorted(os.listdir("downloaded_images"))
i=1;
for file in files:
    if file.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        
        imgloc = os.path.join("downloaded_images", file)
        img = Image.open(imgloc)
        imagev.append((img, file))
# #         plt.figure(figsize=(80,5))
# #         plt.subplot(1,16,i)
# #         plt.imshow(img)
        


In [ ]:
match=[]
unmatch=[]
k=0

for i in range(len(imagev)):
    
    for j in range(len(imagev)):
        k=k+1
        if(i>0 and j>i):
            if(imagev[i][1][:10]==imagev[j][1][:10]):
                match.append((imagev[i][0],imagev[j][0]))
            elif(len(imagev[i][1])>10 and len(imagev[j][1])>10 and k%300==0) :
                unmatch.append((imagev[i][0],imagev[j][0]))
    
            
    

# for x,y in unmatch:
#     plt.subplot(1,2,1)
#     plt.imshow(x)
#     plt.subplot(1,2,2)
#     plt.imshow(y)
#     plt.show()
len(unmatch)

In [ ]:

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [ ]:
#      # Convert PIL images to numpy arrays
#     x2 = np.array(x1)
#     y2 = np.array(y1)
    
#     # Convert images to grayscale for face detection
#     x3 = cv2.cvtColor(x2, cv2.COLOR_BGR2GRAY)
#     y3 = cv2.cvtColor(y2, cv2.COLOR_BGR2GRAY)
    
#     # Detect faces in x3
#     faces_x = face_cascade.detectMultiScale(x3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_x:
#         x4 = x2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Detect faces in y3
#     faces_y = face_cascade.detectMultiScale(y3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_y:
#         y4 = y2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Resize the cropped face regions to (224, 224)
#     x5 = cv2.resize(x4, (224, 224))
#     y5 = cv2.resize(y4, (224, 224))

    
#     # Convert PIL images to numpy arrays
#     x2 = np.array(x1)
#     y2 = np.array(y1)
    
#     # Convert images to grayscale for face detection
#     x3 = cv2.cvtColor(x2, cv2.COLOR_BGR2GRAY)
#     y3 = cv2.cvtColor(y2, cv2.COLOR_BGR2GRAY)
    
#     # Detect faces in x3
#     faces_x = face_cascade.detectMultiScale(x3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_x:
#         x4 = x2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Detect faces in y3
#     faces_y = face_cascade.detectMultiScale(y3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_y:
#         y4 = y2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Resize the cropped face regions to (224, 224)
#     x5 = cv2.resize(x4, (224, 224))
#     y5 = cv2.resize(y4, (224, 224))


In [ ]:
plt.imshow(match[4][0])


In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np

detector = MTCNN()

X = []
Y = []

for x, y in match:
    # Convert PIL images to numpy arrays
    x2 = np.array(x)
    y2 = np.array(y)

    # Convert images to grayscale for face detection
    x3 = cv2.cvtColor(x2, cv2.COLOR_BGR2GRAY)
    y3 = cv2.cvtColor(y2, cv2.COLOR_BGR2GRAY)

    # Convert grayscale images back to RGB
    x_rgb = cv2.cvtColor(x3, cv2.COLOR_GRAY2RGB)
    y_rgb = cv2.cvtColor(y3, cv2.COLOR_GRAY2RGB)

    # Detect faces in x_rgb
    faces_x = detector.detect_faces(x_rgb)
    if faces_x:
        x4 = x2[faces_x[0]['box'][1]:faces_x[0]['box'][1]+faces_x[0]['box'][3],
               faces_x[0]['box'][0]:faces_x[0]['box'][0]+faces_x[0]['box'][2]]  # Crop the face region
        x5 = cv2.resize(x4, (224, 224))  # Resize the cropped face region
    else:
        continue  # Skip if no face detected

    # Detect faces in y_rgb
    faces_y = detector.detect_faces(y_rgb)
    if faces_y:
        y4 = y2[faces_y[0]['box'][1]:faces_y[0]['box'][1]+faces_y[0]['box'][3],
               faces_y[0]['box'][0]:faces_y[0]['box'][0]+faces_y[0]['box'][2]]  # Crop the face region
        y5 = cv2.resize(y4, (224, 224))  # Resize the cropped face region
    else:
        continue  # Skip if no face detected

    # Append to lists
    X.append([x5, y5])
    Y.append(1)

for x, y in unmatch:
    # Convert PIL images to numpy arrays
    x2 = np.array(x)
    y2 = np.array(y)

    # Convert images to grayscale for face detection
    x3 = cv2.cvtColor(x2, cv2.COLOR_BGR2GRAY)
    y3 = cv2.cvtColor(y2, cv2.COLOR_BGR2GRAY)

    # Convert grayscale images back to RGB
    x_rgb = cv2.cvtColor(x3, cv2.COLOR_GRAY2RGB)
    y_rgb = cv2.cvtColor(y3, cv2.COLOR_GRAY2RGB)

    # Detect faces in x_rgb
    faces_x = detector.detect_faces(x_rgb)
    if faces_x:
        x4 = x2[faces_x[0]['box'][1]:faces_x[0]['box'][1]+faces_x[0]['box'][3],
               faces_x[0]['box'][0]:faces_x[0]['box'][0]+faces_x[0]['box'][2]]  # Crop the face region
        x5 = cv2.resize(x4, (224, 224))  # Resize the cropped face region
    else:
        continue  # Skip if no face detected

    # Detect faces in y_rgb
    faces_y = detector.detect_faces(y_rgb)
    if faces_y:
        y4 = y2[faces_y[0]['box'][1]:faces_y[0]['box'][1]+faces_y[0]['box'][3],
               faces_y[0]['box'][0]:faces_y[0]['box'][0]+faces_y[0]['box'][2]]  # Crop the face region
        y5 = cv2.resize(y4, (224, 224))  # Resize the cropped face region
    else:
        continue  # Skip if no face detected

    # Append to lists
    X.append([x5, y5])
    Y.append(-1)

# Convert lists to numpy arrays
X_me = np.array(X)
Y_me = np.array(Y)


In [ ]:


# # Initialize face detector
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# X = []
# Y = []

# for x, y in match:
   
    
#     # Resize PIL images
#     x1 = x.resize((224, 224))
#     y1 = y.resize((224, 224))
    
#     # Convert PIL images to numpy arrays
#     x2 = np.array(x1)
#     y2 = np.array(y1)
    
#     # Convert images to grayscale for face detection
#     x3 = cv2.cvtColor(x2, cv2.COLOR_BGR2GRAY)
#     y3 = cv2.cvtColor(y2, cv2.COLOR_BGR2GRAY)
    
#     # Detect faces in x3
#     faces_x = face_cascade.detectMultiScale(x3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_x:
#         x4 = x2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Detect faces in y3
#     faces_y = face_cascade.detectMultiScale(y3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_y:
#         y4 = y2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Resize the cropped face regions to (224, 224)
#     x5 = cv2.resize(x4, (224, 224))
#     y5 = cv2.resize(y4, (224, 224))

#     # Append to lists
#     X.append([x5, y5])
#     Y.append(1)

# for x, y in unmatch:
#     # Resize PIL images
#     x1 = x.resize((224, 224))
#     y1 = y.resize((224, 224))
#      # Convert PIL images to numpy arrays
#     x2 = np.array(x1)
#     y2 = np.array(y1)
    
#     # Convert images to grayscale for face detection
#     x3 = cv2.cvtColor(x2, cv2.COLOR_BGR2GRAY)
#     y3 = cv2.cvtColor(y2, cv2.COLOR_BGR2GRAY)
    
#     # Detect faces in x3
#     faces_x = face_cascade.detectMultiScale(x3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_x:
#         x4 = x2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Detect faces in y3
#     faces_y = face_cascade.detectMultiScale(y3, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     for (a, b, w, h) in faces_y:
#         y4 = y2[b:b + h, a:a + w]  # Crop the face region
#         break  # Assuming only the first detected face is needed

#     # Resize the cropped face regions to (224, 224)
#     x5 = cv2.resize(x4, (224, 224))
#     y5 = cv2.resize(y4, (224, 224))

#     # Append to lists
#     X.append([x5, y5])
    
#     Y.append(-1)

# # Convert lists to numpy arrays




In [ ]:
# Convert lists to numpy arrays
X_me = np.array(X)
Y_me = np.array(Y)
X_me.shape

In [ ]:
X_train,X_test,Y_train,Y_test=model_selection.train_test_split(X_me,Y_me,test_size=0.3)

In [ ]:
X_train.shape

In [ ]:
len(unmatch)

In [ ]:
## making triplet for triplet loss layer
# k=0
# triplet=[]
# for i in range(len(imagev)):
#     temp=[]
#     temp.append((imagev[i][0]))
#     for j in range(len(imagev)):
#         k=k+1
#         if(i>0 and j>i):
#             if(imagev[i][1][:10]==imagev[j][1][:10] and len(temp)==1):
#                 temp.append((imagev[j][0]))
#             if(imagev[i][1][:10]!=imagev[j][1][:10] and len(temp)==2) :
#                 temp.append((imagev[j][0]))
#                 triplet.append(temp)
#                 break
    
            



In [ ]:
    

# for x,y,z in triplet:
#     plt.subplot(1,3,1)
#     plt.imshow(x)
#     plt.subplot(1,3,2)
#     plt.imshow(y)
#     plt.subplot(1,3,3)
#     plt.imshow(z)
#     plt.show()

In [ ]:
# anc_in=[]
# pos_in=[]
# neg_in=[]
# target=(224,224)
# for x,y,z in triplet:
#     anc_resize=x.resize(target)
#     anc_in.append(anc_resize)
#     pos_resize=y.resize(target)
#     pos_in.append(pos_resize)
#     neg_resize=z.resize(target)
#     neg_in.append(neg_resize)


In [ ]:
# len(triplet)

In [ ]:
Img_shape=[224,224,3]

In [ ]:
img_path="g.jpg"
img=Image.open(img_path)
img_ar=np.array(img)
a=img.resize((224,224))
a

In [ ]:
vgg=VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
# def triplet_loss(y_true,y_pred,alpha=0.5):
#     anc_o,pos_o,neg_o=y_pred[0],y_pred[1],y_pred[2]
#     pos=tf.reduce_mean(tf.square(anc_o-pos_o))
#     neg=tf.reduce_mean(tf.square(anc_o-neg_o))
#     loss1=pos-neg
#     loss=tf.maximum(loss1+alpha,0)
    
#     return loss

In [ ]:
def cosine_similarity_loss(y_true,y_pred):
    loss=tf.abs(y_true-y_pred)
    
    return loss

In [ ]:
# input_shape=(224,224,3)
 
# anchor_input=Input(shape=input_shape,name='input for anchor')
# positive_input=Input(shape=input_shape,name='input for positive image')
# negative_input=Input(shape=input_shape,name='input for negative image')

# anchor_output_vgg=Flatten()(vgg(anchor_input))
# positive_output_vgg=Flatten()(vgg(positive_input))
# negative_output_vgg=Flatten()(vgg(negative_input))

# last_dense=Dense(128,activation='relu')
# anc_op=last_dense(anchor_output_vgg)
# pos_op=last_dense(positive_output_vgg)
# neg_op=last_dense(negative_output_vgg)



In [ ]:
# one more thing you need to add triplet loss function bro


In [ ]:


# Example input shape (replace with your actual input shape)
input_shape = (224, 224, 3)

# Example input layers
one_input = Input(shape=input_shape, name='input_one')
two_input = Input(shape=input_shape, name='input_two')


# Example outputs from VGG16
output_vgg_one = Flatten()(vgg(one_input))
output_vgg_two = Flatten()(vgg(two_input))

# Example dense layers
lt_dense = Dense(128)

# Applying dense layers
first_in = lt_dense(output_vgg_one)
second_in = lt_dense(output_vgg_two)

# Custom similarity metric wrapped in Lambda layer
def similarity_metric(inputs):
    first_in, second_in = inputs
    dot_product = tf.reduce_sum(first_in * second_in, axis=-1)
    norm_first = tf.norm(first_in, axis=-1)
    norm_second = tf.norm(second_in, axis=-1)
    similarity = dot_product / (norm_first * norm_second)
    return similarity

out_before_loss = Lambda(similarity_metric, name='similarity_metric')([first_in, second_in])

# Construct the model
model_c = Model(inputs=[one_input, two_input], outputs=out_before_loss)

# Print model summary for verification
model_c.summary()


In [ ]:

# def my_accuracy(y_true, y_pred):
#     # Cast y_true to float32 if necessary (assuming y_true is int64)
#     y_true = tf.cast(y_true, tf.float32)
    
#     # Check if signs of y_true and y_pred are the same
# #     correct_predictions = tf.equal(tf.sign(y_true), tf.sign(y_pred))
#     total=0.0
#     error=0.0
# #     y_true=np.array(y_true)
# #     y_pred=np.array(y_pred)
#     for a,b in (y_true,y_pred):
#         if(tf.abs(a-b)>=1):
#             error=error+1
#         total=total+1
#     return (1.0-error/total)
            
    
#     # Cast boolean to float and take the mean
#     return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))


In [ ]:



def my_accuracy(y_true, y_pred):
    y_true=tf.cast(y_true,tf.float32)
    for i in range(20):
        if(tf.abs(y_true[i]-y_pred[i])>=1):
            error=error+1
    return error/20




In [ ]:
model_c.compile(optimizer='adam',loss='mse')
model_c.summary()

In [ ]:
X_train1=X_train[:,0]
X_train1.shape
X_train2=X_train[:,1]
Y_train=Y_train.reshape(-1,1)
X_test1=X_test[:,0]
X_test2=X_test[:,1]
Y_test=Y_test.reshape(-1,1)
Y_train

In [ ]:
for x,y,z in zip(X_test1,X_test2,Y_test):
    plt.subplot(1,2,1)
    plt.imshow(x)
    plt.subplot(1,2,2)
    plt.imshow(y)
    print(z)
    plt.show()

In [ ]:
input_cosine=([X_train1,X_train2])
test_cosine=([X_test1,X_test2])

In [ ]:
model_c.fit(input_cosine,Y_train,epochs=20,batch_size=20)

In [ ]:
c=model_c.predict(test_cosine)

In [ ]:
error=0
total=0;
for i in range(len(Y_test)):
    if(tf.abs(c[i]-Y_test[i])>=1):
        error=error+1
    total=total+1
    

In [ ]:
print(1-error/total)

In [ ]:
my_accuracy(Y_test,c)

In [ ]:
c

In [ ]:
model=Model(inputs=[anchor_input,positive_input,negative_input],outputs=[anc_op,pos_op,neg_op])
model.compile(optimizer='adam',loss= triplet_loss)
model.summary()

In [ ]:
anc_in_ar=np.array(anc_in)
pos_in_ar=np.array(pos_in)
neg_in_ar=np.array(neg_in)
inputs=([anc_in_ar,pos_in_ar,neg_in_ar])
dummy_targets = np.zeros((369,))

In [ ]:
model.fit(inputs,dummy_targets,epochs=100,batch_size=5)

In [ ]:
anc1=anc_in_ar[0].reshape(1,224,224,3)
anc1

In [ ]:
last_dense(Flatten()(vgg(anc1)))

In [ ]:
inp=np.array(inputs)
inp1=inp[:,1,:,:,:].reshape(3,1,224,224,3)
inp2=inp1.tolist()

In [ ]:
model.predict(inp1)

In [ ]:
input_ar=np.array(inputs)

In [ ]:
image=Image.fromarray(input_ar[0][0])
image